In [1]:
# author: Yin 
# 08 13 simulation prediction, do not use panel feature
#4 27 2020, change panel identifier from id to csid, get identical results as LG
# 05 04 2020 edefine scaled utility model (0427 folder's model keep original starting values)
# 05 05 test new scale starting values: all 0 except mu=5 as starting values give the best results  04logit_rrm~09.html
# 05 05 also tested mu*(sum of covar) form, results are bad, plus not make sense: (1) can enlarge coeff w/o introducing mu;
#       (2) hard to interpret mu and mu and other effects are confunded

# RRM with panel data and scale variables
# Three alternatives: sq,a,b; 

In [2]:
import sys
import pandas as pd
import biogeme.database as db
import biogeme.biogeme as bio
import biogeme.models as models
import biogeme.results as res
from biogeme.expressions import *
import biogeme.expressions as ex

#import machine learning package
from sklearn.metrics import make_scorer, confusion_matrix, classification_report,recall_score, f1_score, accuracy_score, precision_score
from sklearn.model_selection import TimeSeriesSplit, KFold, ShuffleSplit, LeavePGroupsOut,StratifiedKFold, GroupShuffleSplit,GroupKFold, StratifiedShuffleSplit
                                     
                                     
 
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
%matplotlib inline

cmap_data = plt.cm.Paired
cmap_cv = plt.cm.RdBu
n_splits = 4

In [3]:
#Calculating confidence intervals for elasticities requires interval arithmetics 
#pip install python-intervals in anaconda prompt
#in case python-intervals doesn't work, also install a newer version "portion"

import portion as p
import intervals as ivs
import interval as iv

In [4]:
print("Running 03logit_rrm_0427.py...")

Running 03logit_rrm_0427.py...


In [5]:
# Read the data
df = pd.read_csv("agree0623_all_0427.csv")
database = db.Database("agree0623_all_0427",df)
# They are organized as panel data. The variable ID identifies each individual.
#database.panel("csid")
# use the names of the variable as Python variable.
globals().update(database.variables)

In [6]:
# Normalize the weights
sumWeight = database.data['weight'].sum()
print(sumWeight) 
##result=7423.735097, sample size=7414

7423.735097


In [7]:
normalizedWeight = weight * 7414 / 7423.735097
# originally weight the whole sample use 1123 cases, 
# then delete nonresponse for choice questions, and 945 cases are used for logit

In [8]:
# Parameters to be estimated 
#NZone variables are already scaled: zone/100
#use 0504~~06 biogeme rum estimates as starting values 
B_NZONE1 = Beta('B_NZONE1',-0.0445,None,None,0)
B_NZONE2 = Beta('B_NZONE2',-0.0228,None,None,0)
B_NZONE3 = Beta('B_NZONE3',-0.0216,None,None,0)
B_TOWNDOWN = Beta('B_TOWNDOWN',0.005,None,None,0)
B_TOWNMID = Beta('B_TOWNMID',0.0604,None,None,0)
B_TOWNUP = Beta('B_TOWNUP',0.118,None,None,0)
B_ECO = Beta('B_ECO',0.0353,None,None,0)
B_REC = Beta('B_REC',0.0059,None,None,0)
B_DRY = Beta('B_DRY',-0.124,None,None,0)
B_TAX = Beta('B_TAX',-0.0157,None,None,0)

# covariates for scales
B_ASCEND = Beta('B_ASCEND',0,None,None,0)
B_DESCEND = Beta('B_DESCEND',0,None,None,0)
B_INCPP50K = Beta('B_INCPP50K',0,None,None,0)
B_INCPP75K = Beta('B_INCPP75K',0,None,None,0)
B_INCPP75KM = Beta('B_INCPP75KM',0,None,None,0)
B_F1FLOW = Beta('B_F1FLOW',0,None,None,0)
B_F3PROTUP = Beta('B_F3PROTUP',0,None,None,0)
B_F4CTH2O = Beta('B_F4CTH2O',0,None,None,0)
B_TAXGRT = Beta('B_TAXGRT',0,None,None,0)
#test a few diff starting values, 5 is a good one
mu    = Beta('mu',5,None,None,0)


In [9]:
# Definition of the utility functions
mu_scale = mu+ B_ASCEND*ascend+B_DESCEND*descend+B_INCPP50K*incpp_50k+B_INCPP75K*incpp_75k+B_INCPP75KM*incpp_75km+B_F1FLOW*f1_flow+B_F3PROTUP*f3_protup+B_F4CTH2O*f4_CTh2o+B_TAXGRT*taxgrt



RSQ = mu_scale * (log( 1 + exp((B_NZONE1/mu_scale) * (nzone1_a-nzone1_sq))) + log(1 + exp((B_NZONE1/mu_scale ) * (nzone1_b-nzone1_sq))) + log(1 + exp((B_NZONE2/mu_scale) * (nzone2_a-nzone2_sq))) + log(1 + exp((B_NZONE2 / mu_scale ) * (nzone2_b-nzone2_sq))) + log(1 + exp((B_NZONE3 / mu_scale ) * (nzone3_a-nzone3_sq))) + log(1 + exp((B_NZONE3 / mu_scale ) * (nzone3_b-nzone3_sq))) + log( 1 + exp((B_TOWNDOWN / mu_scale ) * (towndown_a-towndown_sq))) + log(1 + exp((B_TOWNDOWN / mu_scale ) * (towndown_b-towndown_sq)))  + log(1 + exp((B_TOWNMID / mu_scale ) * (townmid_a-townmid_sq))) + log(1 + exp((B_TOWNMID / mu_scale ) * (townmid_b-townmid_sq))) + log(1 + exp((B_TOWNUP / mu_scale ) * (townup_a-townup_sq))) + log(1 + exp((B_TOWNUP / mu_scale ) * (townup_b-townup_sq))) + log( 1 + exp((B_ECO / mu_scale ) * (eco_a-eco_sq))) + log(1 + exp((B_ECO / mu_scale ) * (eco_b-eco_sq))) + log(1 + exp((B_REC / mu_scale ) * (rec_a-rec_sq))) + log(1 + exp((B_REC / mu_scale ) * (rec_b-rec_sq))) + log(1 + exp((B_DRY / mu_scale ) * (dry_a-dry_sq))) + log(1 + exp((B_DRY / mu_scale ) * (dry_b-dry_sq))) + log(1 + exp((B_TAX / mu_scale ) * (tax_a-tax_sq))) + log(1 + exp((B_TAX / mu_scale ) * (tax_b-tax_sq))))                                                
          
RA  = mu_scale * (log( 1 + exp((B_NZONE1/mu_scale) * (nzone1_sq-nzone1_a))) + log(1 + exp((B_NZONE1/mu_scale ) * (nzone1_b-nzone1_a))) + log(1 + exp((B_NZONE2/mu_scale) * (nzone2_sq-nzone2_a))) + log(1 + exp((B_NZONE2 / mu_scale ) * (nzone2_b-nzone2_a))) + log(1 + exp((B_NZONE3 / mu_scale ) * (nzone3_sq-nzone3_a))) + log(1 + exp((B_NZONE3 / mu_scale ) * (nzone3_b-nzone3_a))) + log( 1 + exp((B_TOWNDOWN / mu_scale ) * (towndown_sq-towndown_a))) + log(1 + exp((B_TOWNDOWN / mu_scale ) * (towndown_b-towndown_a))) + log(1 + exp((B_TOWNMID / mu_scale ) * (townmid_sq-townmid_a))) + log(1 + exp((B_TOWNMID / mu_scale ) * (townmid_b-townmid_a))) + log(1 + exp((B_TOWNUP / mu_scale ) * (townup_sq-townup_a))) + log(1 + exp((B_TOWNUP / mu_scale ) * (townup_b-townup_a))) + log( 1 + exp((B_ECO / mu_scale ) * (eco_sq-eco_a))) + log(1 + exp((B_ECO / mu_scale ) * (eco_b-eco_a))) + log(1 + exp((B_REC / mu_scale ) * (rec_sq-rec_a))) + log(1 + exp((B_REC / mu_scale ) * (rec_b-rec_a))) + log(1 + exp((B_DRY / mu_scale ) * (dry_sq-dry_a))) + log(1 + exp((B_DRY / mu_scale ) * (dry_b-dry_a))) + log(1 + exp((B_TAX / mu_scale ) * (tax_sq-tax_a))) + log(1 + exp((B_TAX / mu_scale ) * (tax_b-tax_a))))   
           
RB  = mu_scale * (log( 1 + exp((B_NZONE1/mu_scale) * (nzone1_sq-nzone1_b))) + log(1 + exp((B_NZONE1/mu_scale ) * (nzone1_a-nzone1_b))) + log(1 + exp((B_NZONE2/mu_scale) * (nzone2_sq-nzone2_b))) + log(1 + exp((B_NZONE2 / mu_scale ) * (nzone2_a-nzone2_b))) + log(1 + exp((B_NZONE3 / mu_scale ) * (nzone3_sq-nzone3_b))) + log(1 + exp((B_NZONE3 / mu_scale ) * (nzone3_a-nzone3_b))) + log( 1 + exp((B_TOWNDOWN / mu_scale ) * (towndown_sq-towndown_b))) + log(1 + exp((B_TOWNDOWN / mu_scale ) * (towndown_a-towndown_b))) + log(1 + exp((B_TOWNMID / mu_scale ) * (townmid_sq-townmid_b))) + log(1 + exp((B_TOWNMID / mu_scale ) * (townmid_a-townmid_b))) + log(1 + exp((B_TOWNUP / mu_scale ) * (townup_sq-townup_b))) + log(1 + exp((B_TOWNUP / mu_scale ) * (townup_a-townup_b))) + log( 1 + exp((B_ECO / mu_scale ) * (eco_sq-eco_b))) + log(1 + exp((B_ECO / mu_scale ) * (eco_a-eco_b))) + log(1 + exp((B_REC / mu_scale ) * (rec_sq-rec_b))) + log(1 + exp((B_REC / mu_scale ) * (rec_a-rec_b))) + log(1 + exp((B_DRY / mu_scale ) * (dry_sq-dry_b))) + log(1 + exp((B_DRY / mu_scale ) * (dry_a-dry_b))) + log(1 + exp((B_TAX / mu_scale ) * (tax_sq-tax_b))) + log(1 + exp((B_TAX / mu_scale ) * (tax_a-tax_b))))   


In [10]:
# Associate utility functions with the numbering of alternatives
V = {1: -RSQ,
     2: -RA,
     3: -RB}

# Associate the availability conditions with the alternatives
# all alternatives are available for each individual
av = {1: 1,
      2: 1,
      3: 1}

In [11]:
# Definition of the model. This is the contribution of each observation to the log likelihood function.
logprob = models.loglogit(V,av,choice)
prob_sq = models.logit(V,av,1)
prob_a = models.logit(V,av,2)
prob_b = models.logit(V,av,3)

simulate = {'simprob_sq':prob_sq,
            'simprob_a':prob_a,
            'simprob_b':prob_b}



biogeme  = bio.BIOGEME(database,simulate)
biogeme.modelName = "03logit_rrm_scale_simu_0813"

In [13]:
biogeme.createLogFile()

results = biogeme.simulate()

# Get the results in a pandas table
#pandasResults = results.getEstimatedParameters()
#print(pandasResults)


print("Results=",results)

Results=       simprob_sq  simprob_a  simprob_b
0       0.215018   0.437310   0.347672
1       0.217412   0.385622   0.396966
2       0.191471   0.398940   0.409589
3       0.155318   0.672025   0.172657
4       0.229513   0.382841   0.387646
...          ...        ...        ...
7409    0.212339   0.339724   0.447936
7410    0.244525   0.326957   0.428517
7411    0.231022   0.534569   0.234408
7412    0.209821   0.295674   0.494505
7413    0.189348   0.401561   0.409092

[7414 rows x 3 columns]


In [14]:
results.loc[(results['simprob_sq']>results['simprob_a']) & (results['simprob_sq']>results['simprob_b']),"choice_simpre"]=1
results.loc[(results['simprob_a']>results['simprob_sq']) & (results['simprob_a']>results['simprob_b']),"choice_simpre"]=2
results.loc[(results['simprob_b']>results['simprob_sq']) & (results['simprob_b']>results['simprob_a']),"choice_simpre"]=3
results['choice_simpre'] = results['choice_simpre'].astype(int)
print(results)

      simprob_sq  simprob_a  simprob_b  choice_simpre
0       0.215018   0.437310   0.347672              2
1       0.217412   0.385622   0.396966              3
2       0.191471   0.398940   0.409589              3
3       0.155318   0.672025   0.172657              2
4       0.229513   0.382841   0.387646              3
...          ...        ...        ...            ...
7409    0.212339   0.339724   0.447936              3
7410    0.244525   0.326957   0.428517              3
7411    0.231022   0.534569   0.234408              2
7412    0.209821   0.295674   0.494505              3
7413    0.189348   0.401561   0.409092              3

[7414 rows x 4 columns]


In [15]:
#a naive test w/o considering the panel structure
y_simpredict = results['choice_simpre']  
y_true= df.choice
print("simulation Accuracy : ", accuracy_score(y_true, y_simpredict))
print("simulation Accuracy : ", accuracy_score( y_simpredict,y_true))

simulation Accuracy :  0.4817912058268141
simulation Accuracy :  0.4817912058268141
